# Gradient Boosting Decision Tree (GBDT)

このハンズオンでは、勾配ブースティング手法のライブラリであるLightGBMを使って、構造化(テーブル)データに対してのモデルを作成していきます。

また勾配ブースティングでは、比較的簡単に予測結果の判断根拠も求めることができます。

この利点としては
- 予測の判断根拠自体が推論システムとしてあると何かと便利
- 学習したモデルが一般的な常識/知見に基づいた特徴量であるか確認できる  

などが挙げられます。

このような、予測結果の判断根拠を得る手法についても紹介していきます。

In [ ]:
# 必要なライブラリ類のインストール
# 2回目以降は実行不要
!pip install lightgbm shap category_encoders storage

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from io import BytesIO
from google.cloud import storage

In [ ]:
## GoogleCloudStorageの接続
project_id = 'hr-mixi'
buclet_name = 'mixi-ml-handson-2023'

client = storage.Client(project_id)
bucket = client.get_bucket(buclet_name)

## データセットの読み込み
今回は、実データを用いたタスクに挑戦していきます。  
具体的には、過去数年の競輪のレースデータを使って、着順予測をしていきます。

<font color='red'>**注意: 今回のデータは非公開データとなります。本研修外での利用はお控えください。**</font>

GCSにデータを置いているので、取得します。

In [ ]:
training_data_path = '05_predict_structured_data/training_data/race_data.csv'

blob = bucket.blob(training_data_path)
content = blob.download_as_string()
df = pd.read_csv(BytesIO(content))

取得できたら、データを観察してみましょう。 

In [ ]:
# データサイズの確認
df.shape

In [ ]:
# データの機関の確認
'日付開始日: {}, 日付終了日: {}'.format(df['KaisaiDate'].max(), df['KaisaiDate'].min())

In [ ]:
# データの統計量の確認
df.describe()

In [ ]:
# 0行目のデータの内容
for c in df.columns:
    print(c, df.iloc[0][c])

**NOTE:** 今回は時間の都合上各カラムのデータをさらっと眺めるだけにしていますが、機械学習モデルを開発する上で、データへの理解度(ドメイン知識)は非常に重要な要素となってきます。  
データのAnalysisだけでも時間をかける価値があるということを理解しておいてください。

## ターゲットデータの作成
次に、データからラベルを作成していきます。  
今回は、KakuteiJyuniつまり着順を使用します。

このとき、着順をそのままラベルに使用する場合、着順それぞれを1クラスとした多クラス分類となります。  
多クラス分類になると、それぞれのクラスを独立したものとして扱うことになりますが、1着と2着などを別クラスとして扱うことは正しいでしょうか。  

例えば工夫した考え方として、ラベルを`3着以内orNot`や`1着orNot`に変換する方法も考えられます。  
このようにした場合、今回のタスクは２値分類タスクとなります。  
また、`1着orNot`にした場合、出力は確率になるので、その確率が高い順と考えると、結果的に着順も予測できることになります。  

ラベルをどのように扱えばより良いかは自分で考える必要があるので、このようなドメイン知識が重要になってきます。  

初回はそのまま着順をそのままラベルに使用した多クラス分類モデルを作成します

<Challenge>
後ほど、各々ここのターゲットを各自変えて再度モデルを学習し直してもらいます   
    
- ヒント: メモリ上に実行した引数が残ってしまっていると意図した動作をしないケースが発生するので、モデルを作り直す場合はカーネルリセットをお勧めします
    
サンプル
・1着orNotの2値分類
```
target = df['KakuteiJyuni'].apply(lambda x: 1 if x == 1 else 0)
```
    
・3着以内orNotの2値分類
```
target = df['KakuteiJyuni'].apply(lambda x: 1 if x <= 3 else 0)
```

In [ ]:
# LightGBMのマルチクラス分類のラベルは0から始まる連番である必要があるので、
# 1着 → 0, 2着 → 1　...のように変換します
target = df['KakuteiJyuni'] - 1

## 学習/テスト用データセットの分割

テストデータは日付で2023/01/01以降のレースを対象とします。

In [ ]:
train_test_split_date = 20230101

train_index = df[df['KaisaiDate'] < train_test_split_date].index
test_index = df[df['KaisaiDate'] >= train_test_split_date].index

df_train = df.loc[train_index].reset_index(drop=True)
target_train = target[train_index].values

df_test = df.loc[test_index].reset_index(drop=True)
target_test = target[test_index].values

## 前処理
前処理では一般的に以下のような処理を行います。  

- 文字列やカテゴリカルなデータを数値やベクトルに変換(LabelEncording, OneHotEncordingなど)  
- アルゴリズムが学習しやすいように変換(欠損値処理、正規化など)  
- 組み合わせや集計処理によって新しい特徴量を作成  
など

前処理コードを学習と予測用で分けているのは、予測時に学習時のラベルエンコーディングのIDのマップと同じようにIDを振り分ける必要がある等、学習時と同様のデータの変換をするように予測用の前処理コードに記載する必要があるためです。  
今回もタスクに必要な前処理を行っていきます。

**\<チャレンジ\>**  
余裕があれば、特徴量の変換や集計で新しい特徴量を作成してみてください。

In [ ]:
# 学習に使用しないカラムのリスト
# ・結果データ・選手/レースを識別してしまうデータを取り除く
unuse_feature_list = [
    'RaceId', 'KaisaiDate', 'SenshuCD', 'Kimari', 'IJyoCD', 'KakuteiJyuni', 'AgariTime', 'ChakusaCD', 'StdGet', 'JanGet', 'HomeGet', 'BackGet',
    'Hondai', 'GaiteiName', 'HassoTime', 'HassoTimeOld1', 'HassoTimeOld2', 'HassoTimeOld3'
]

# レース内の集計をする特徴量
stat_columns = [
    'Age', 'Gear', 'Graduate', 'RaceRating', 'Shokin'
]

# カテゴリカル特徴量のリスト
categorical_feature_list = [
    # 今回レースに関する特徴量
    'JyoCD', 'KaisaiGrade', 'Kyori',
    # 選手に関する特徴量
    'Kyu', 'Han', 'Fuken', 'Kyakushitu',
    # 1走前レースに関する特徴量
    'JyoCDOld1', 'RaceNumOld1', 'HondaiOld1', 'GaiteiNameOld1','KyoriOld1',
    # 1走前選手に関する特徴量
    'SyabanOld1', 'ChakusaCDOld1', 'StdGetOld1', 'JanGetOld1', 'HomeGetOld1', 'BackGetOld1',
    # 2走前レースに関する特徴量
    'JyoCDOld2', 'RaceNumOld2', 'HondaiOld2', 'GaiteiNameOld2','KyoriOld2',
    # 2走前選手に関する特徴量
    'SyabanOld2', 'ChakusaCDOld2', 'StdGetOld2', 'JanGetOld2', 'HomeGetOld2', 'BackGetOld2',
    # 3走前レースに関する特徴量
    'JyoCDOld3', 'RaceNumOld3', 'HondaiOld3', 'GaiteiNameOld3','KyoriOld3',
    # 3走前選手に関する特徴量
    'SyabanOld3', 'ChakusaCDOld3', 'StdGetOld3', 'JanGetOld3', 'HomeGetOld3', 'BackGetOld3',
    # 決まり手に関する特徴量
    'KimariOld1', 'KimariOld2', 'KimariOld3'
]

import category_encoders as ce


def training_preprocessing(df):
    # カテゴリカル変数の処理
    # LightGBMでは、ラベルエンコーディング(カテゴリをintでID付け)して、typeをcategoryに変換すればライブラリが上手く扱ってくれる
    categorical_encorder = ce.OrdinalEncoder(cols=categorical_feature_list, handle_unknown='impute')
    
    df = categorical_encorder.fit_transform(df)
        
    # レース内での平均値の特徴量の作成
    race_mean = df.groupby(['RaceId'])[stat_columns].mean()

    race_mean = race_mean.rename(columns= {x: x+'Mean' for x in race_mean.columns})
    df = pd.merge(df, race_mean, how='left', on=['RaceId'])
    
    # レース内での中央値の特徴量の作成
    race_median = df.groupby(['RaceId'])[stat_columns].median()

    race_median = race_median.rename(columns= {x: x+'Median' for x in race_median.columns})
    df = pd.merge(df, race_median, how='left', on=['RaceId'])
    
    # 不要なカラムを削除する
    df = df.drop(unuse_feature_list, axis=1)

    return df, categorical_encorder


def prediction_preprocessing(df, categorical_encorder):
    # カテゴリカル変数の処理
    df = categorical_encorder.transform(df)
        
    # レース内での平均値の特徴量の作成
    race_mean = df.groupby(['RaceId'])[stat_columns].mean()

    race_mean = race_mean.rename(columns= {x: x+'Mean' for x in race_mean.columns})
    df = pd.merge(df, race_mean, how='left', on=['RaceId'])
    
    # レース内での中央値の特徴量の作成
    race_median = df.groupby(['RaceId'])[stat_columns].median()

    race_median = race_median.rename(columns= {x: x+'Median' for x in race_median.columns})
    df = pd.merge(df, race_median, how='left', on=['RaceId'])
    
    # 不要なカラムを削除する
    df = df.drop(unuse_feature_list, axis=1)

    return df


In [ ]:
# 前処理の実行には1分程度かかります
preproessed_df_train, categorical_encorder = training_preprocessing(df_train)

In [ ]:
preproessed_df_train

## 学習データから検証データを分割

学習用と検証用は8:2で分割しています。  
検証用データは、モデル学習時のtest_lossの計算のほか、  
パラメーターの自動チューニング時のloss指標の計算に使用されます。

In [ ]:
val_rate = 0.2

train_val_split_point = int(len(df_train)*(1-val_rate))

preproessed_df_val = preproessed_df_train.iloc[train_val_split_point:].reset_index(drop=True)
preproessed_df_train = preproessed_df_train.iloc[:train_val_split_point].reset_index(drop=True)

target_val = target_train[train_val_split_point:]
target_train = target_train[:train_val_split_point]

## 学習

準備が整ったので、モデルの学習をさせていきます。

主要なLightGBMのパラメータを下記の通りです。

```
- 出力形式に関する要素
    - objective
        - regression: 回帰
        - binary: 二値分類
        - multiclass: 多クラス分類
    - metric
        - 回帰
            - mae: mean absolute error: 平均絶対誤差
            - mse: mean squared error: 平均2乗誤差
        - 二値分類
            - binary_logloss:　クロスエントロピー
            - binary_error: 正解率
        - 多クラス分類
            - multi_logloss: softmax
            - multi_error: 正解率
- モデル構造に関する要素
    - learning_rate: 学習率 Default=0.1 0以上
    - num_iterations: 木の数
    - num_leaves: 葉(条件の数)
    - max_depth: 木の深さの最大値
```

公式リファレンス　https://lightgbm.readthedocs.io/en/latest/Parameters.html
  
**\<チャレンジ\>** 
余裕が有れば、lgb_paramsに任意のパラメータを追加して、モデルの精度の変化を観察してください

In [ ]:
# 着順の多クラス分類モデルの学習

lgb_train = lgb.Dataset(preproessed_df_train, target_train)
lgb_test = lgb.Dataset(preproessed_df_val, target_val, reference=lgb_train)

## <TODO> lgb_paramsのobjectiveとmetricとnum_classを正しい値で埋めてください
lgb_params = {
    'boosting_type': 'gbdt',
    'n_estimators': 1000,
    'num_class': ______,
    'early_stopping_rounds': 50,
    'objective': '______',
    'metric': '______',
}

booster = lgb.train(
    lgb_params, lgb_train, valid_sets=lgb_test
)

学習が始まって、経過が確認できるでしょうか

## モデルの判断根拠: Importance

学習が完了したら、特徴量毎のImportanceを確認してみましょう。  
feature_importanceメソッドでモデルにおける特徴量の重要度を得られます。  
重要度の指標:
- split: 決定木の分岐を使用した数  
- gain: その特徴量が使用する分岐からの目的関数の減少  

In [ ]:
# テストデータの前処理の実行
preprocessed_df_test = prediction_preprocessing(df_test, categorical_encorder)

In [ ]:
# split
importance = pd.DataFrame(
    booster.feature_importance(importance_type = 'split'),
    index=preproessed_df_train.columns,
    columns=['importance']
)
importance.sort_values(['importance'], ascending=False)

In [ ]:
# gain
importance = pd.DataFrame(
    booster.feature_importance(importance_type = 'gain'),
    index=preproessed_df_train.columns,
    columns=['importance']
)
importance.sort_values(['importance'], ascending=False)

## モデルの判断根拠: SHAP
また、SHAPを用いることでも、特徴量がモデル/予測に対してどのような影響を与えたかを計測できます。  
試してみましょう。

In [ ]:
import shap
shap.initjs()

In [ ]:
#LightGBMは決定木アルゴリズムなのでTreeExplainerを使います
# NNモデルでDeepExplainerを使うと今回のようなテーブルデータだけでなく、画像データに対しても適用することができます
# ２分程度時間がかかります

explainer = shap.TreeExplainer(booster, feature_perturbation = "tree_path_dependent")
shap_values = explainer.shap_values(preprocessed_df_test.values)

In [ ]:
# 特徴量の貢献度をプロット
shap.summary_plot(shap_values, preprocessed_df_test, plot_type="bar")

### 予測ごとの判断根拠
shap_valuesには各予測に対しての各特徴量の貢献度が格納されています。  
これを使うことで、例えばあるレースの結果の予測は、この特徴量が強く影響しているためという表現ができるようになります。  

検証用データの0番目の予測結果を見てみましょう。

In [ ]:
idx = 0

shap_v = pd.DataFrame(shap_values[1], columns=preprocessed_df_test.columns)

In [ ]:
# 予測結果に対しでpositiveな影響を与えている特徴量
shap_v.iloc[idx].sort_values(ascending=False).iloc[:10]

In [ ]:
# 予測結果に対しでnegativeな影響を与えている特徴量
shap_v.iloc[idx].sort_values(ascending=True).iloc[:10]

## 精度検証

特徴量の貢献度はあらかた調べられたので、締め括りとしてこのモデルの精度の評価をしましょう。
今回の精度指標は、  
`レース内で最も1着である確率である確率が高いと評価された選手が実際に1着であった確率`  
と設定します。

データは１行で1選手を表現しているため、1レース単位は複数行を集約する必要があります。  
groupbyを使ってレースごとのindexを取得していきましょう。

In [ ]:
race_groups = df_test.groupby(['RaceId']).groups

### 着順の多クラス分類モデル

In [ ]:
# 予測結果の取得
prediction_result = booster.predict(preprocessed_df_test)

In [ ]:
# 予測結果の0番目の出力
# 9つの要素がありそれぞれが各ラベルの確率を表現しています、つまり合計は1になります

# Challengeで目的変数を2値分類にした場合出力される予測結果の形変わります
print(prediction_result[0])

In [ ]:
# レースごとの出力値の0番目(1着と推定した確率)がもっとも大きい(np.argmax)選手の実際の着順が1着だった数/レースの数
[
    df_test.iloc[idx[np.argmax(
        [
            res[0] for res in prediction_result[idx]
        ]
    )]]['KakuteiJyuni']
    for race, idx in race_groups.items()
].count(1) / len(race_groups)

========================================================================

# やってみよう: リアルタイムの競輪レース予測

ここまでで、GBDTのハンズオンは一通り終了です。  
ですが、折角競輪の予測を作ったので、実際に今日のレースを予測してみましょう！

GCSに本日のレースデータのcsvを作成してあります。  
このデータに対する予測結果を作成して、今日の全レースに対する予測を作成していきましょう。

In [ ]:
# 本日のレースデータをGCSから取得
prediction_data_path = 'prediction_data/race_data.csv'
blob = bucket.blob(prediction_data_path)
content = blob.download_as_string()
prediction_df = pd.read_csv(BytesIO(content))

In [ ]:
# groupsの作成
prediction_race_groups = prediction_df.groupby(['JyoCD', 'RaceNum']).groups

# 前処理の実行
preprocessed_prediction_df = prediction_preprocessing(prediction_df, categorical_encorder)

# 予測結果の作成
today_prediction_result = booster.predict(preprocessed_prediction_df)

できたら、今日のレースが予想とマッチするかみていきます。

### 予測結果の確認
まず、今日の全レースの予想を出力します。

In [ ]:
velodrome_code = {
    11: '函館競輪場', 
    12: '青森競輪場', 
    13: 'いわき平競輪場', 
    21: '弥彦競輪場', 
    22: '前橋競輪場', 
    23: '取手競輪場', 
    24: '宇都宮競輪場', 
    25: '大宮競輪場', 
    26: '西武園競輪場', 
    27: '京王閣競輪場', 
    28: '立川競輪場', 
    31: '松戸競輪場', 
    32: '千葉競輪場', 
    33: '花月園競輪場', 
    34: '川崎競輪場', 
    35: '平塚競輪場', 
    36: '小田原競輪場', 
    37: '伊東競輪場', 
    38: '静岡競輪場', 
    41: '一宮競輪場', 
    42: '名古屋競輪場', 
    43: '岐阜競輪場', 
    44: '大垣競輪場', 
    45: '豊橋競輪場', 
    46: '富山競輪場', 
    47: '松阪競輪場', 
    48: '四日市競輪場', 
    51: '福井競輪場', 
    52: '大津競輪場', 
    53: '奈良競輪場', 
    54: '向日町競輪場', 
    55: '和歌山競輪場', 
    56: '岸和田競輪場', 
    61: '玉野競輪場', 
    62: '広島競輪場', 
    63: '防府競輪場', 
    71: '高松競輪場', 
    72: '観音寺競輪場', 
    73: '小松島競輪場', 
    74: '高知競輪場', 
    75: '松山競輪場', 
    81: '小倉競輪場', 
    83: '久留米競輪場', 
    84: '武雄競輪場', 
    85: '佐世保競輪場', 
    86: '別府競輪場',
    87: '熊本競輪場', 
}

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# コールバック関数を定義する
def on_button_clicked(b):
    output.clear_output()
    with output:
        race_df = prediction_df[(prediction_df['JyoCD'] == dropdown1.value)&(prediction_df['RaceNum'] == dropdown2.value)]
        prediction_result = today_prediction_result[race_df.index]
        normed_result = prediction_result / sum(prediction_result)
        
        df = pd.DataFrame({
            '車番': race_df['Syaban'].values,
            '予測勝率': [str(round(prob[0] * 100, 1))+'%' for prob in normed_result]
        })
        # DataGridを作成する
        grid = widgets.GridBox(
            [widgets.HTML(value=df.to_html(index=False, border=1).replace('<table', '<table cellspacing=0'))],
            layout=widgets.Layout(grid_template_columns="repeat(3, 100px)")
        )

        # DataGridを表示する
        display(grid)

        
# ドロップダウンウィジェットを作成する
jyo_list = [(velodrome_code[v], v) for v in prediction_df['JyoCD'].unique()]
race_num_list = prediction_df['RaceNum'].unique()
dropdown1 = widgets.Dropdown(options=jyo_list, description='場: ')
dropdown2 = widgets.Dropdown(options=race_num_list, description='レーズ番号')

# ボタンウィジェットを作成する
button = widgets.Button(description='Submit')
output = widgets.Output()

# ボタンクリック時にコールバック関数を呼び出す
button.on_click(on_button_clicked)

# ウィジェットを表示する
display(dropdown1, dropdown2, button, output)

予測結果を出力できたら、その予測が当たってるかどうかをTipstarで確認してみましょう。  
https://tipstar.com/keirin/channels  
過去のレースは既に結果が出ているので、答え合わせができるかと思います。

また競輪は比較的高頻度で開催されているので、今の時間帯でもレース開始時間の近いものがあると思います。  
上記のURLから、直近のレースを選択してください。
そして、そのレースの予測を今一度確認してみてください。    
確認できたら、予測とレース結果が同じになることをリアルタイムで確認していきます。  
Tipstarにレースの映像があるので、それを見ながら予想した選手を応援しましょう！